In [1]:
# Importing the Requirments
import pandas as pd
import numpy as np
import dash
import dash_table
from datetime import datetime
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [224]:
# Selecting the Desired Columns
b_cols = ['URL', 'Code', 'Symbol', 'Name', 'Date', 'Time', 'Open', 'Final', 'Close', 'Count', 'Volume', 'Value',
          'Lowest', 'Highest', 'Yesterday', 'EPS', 'Base Volume', 'UNKNOWN', 'DROPPED', 'Industry Code', 'Max',
          'Min', 'Share Count', 'DROPPED', 'Sell 1st Count', 'Buy 1st Count', 'Buy 1st Price', 'Sell 1st Price',
          'Buy 1st Volume', 'Sell 1st Volume', 'DROPPED', 'Sell 2nd Count', 'Buy 2nd Count', 'Buy 2nd Price',
          'Sell 2nd Price', 'Buy 2nd Volume', 'Sell 2nd Volume', 'DROPPED', 'Sell 3rd Count', 'Buy 3rd Count', 
          'Buy 3rd Price', 'Sell 3rd Price', 'Buy 3rd Volume', 'Sell 3rd Volume']

col_idx = [0,2,4,6,8,12,13]
col_name = [b_cols[item] for item in col_idx]

# Setting the Basic Path
b_path = 'http://cdn.tsetmc.com/tsev2/data/MarketWatchPlus.aspx?i='

# Maturity DataFrames
mat = {'URL':[18067743398074470,55963441389800122,71901111301490182],
       'Maturity': [datetime(2021,6,28),datetime(2021,8,9),datetime(2021,8,16)]}
matdf = pd.DataFrame(mat).set_index('URL')

In [225]:
# Yield-To-Maturity Function
def YTM(URL, type='Close'):
    n = matdf.loc[URL]-final_df.loc[URL]['Date']
    percent = ((1000000/final_df.loc[URL][type])**(365/n[0].days))-1
    return percent*100

In [228]:
# Initiating the App
app = dash.Dash()

# Designing the Layout
app.layout = html.Div([
    
    html.Div([
        dash_table.DataTable(
            id='MainTable',
            columns=[{"name": i, "id": i} for i in col_name+['YTM']],
        )],
        style={'width': '30%', 'display': 'inline-block'}
    ),
    
    html.P(),
    
    html.Div([
        dcc.Dropdown(
            id='TargetSelector',
            options=[
                {'label': 'Akhza-713', 'value': '55963441389800122'},
                {'label': 'Akhza-716', 'value': '18067743398074470'},
                {'label': 'Akhza-718', 'value': '71901111301490182'}
            ],
            multi=True,
            placeholder='Select the Target(s)'
        )], 
        style={'width': '20%', 'display': 'inline-block'}
    ),
    
    dcc.Interval(
        id='IntervalTimer',
        interval=6000,
        n_intervals=0
    )
   
])




# Interacting with the Layout

# Selecting the Targets
@app.callback(Output('MainTable', 'data'),
              [Input('IntervalTimer', 'n_intervals'),
               Input('TargetSelector', 'value')])
def target_selector(n, targets):
    # Downloading the Data
    b_datasets = []
    for item in targets:
        b_datasets.append(pd.read_csv(b_path+item, header=None))
        
    # Making the Basic Dataframe
    b_df = pd.concat(b_datasets)
    b_df = b_df[col_idx]
    
    # Making the Final Dataframe
    final_df = b_df
    final_df.columns = col_name
    final_df = final_df.set_index('URL')
    final_df['Date'] = pd.to_datetime(final_df['Date'], format='%Y%m%d')
    final_df['YTM'] = [YTM(item) for item in final_df.index]
    
    return final_df.to_dict('records')




# Initiating the Server
if __name__ == '__main__':
    app.run_server()